Load Data

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression

train_u = np.load("u_train.npy")
train_y = np.load("output_train.npy")
test_u = np.load("u_test.npy")

Creation of X matrix

In [2]:
import numpy as np

def X_matrix_creation(n,m,d,train_u,train_y):

    p = max(n, m + d)

    N = len(train_u)

    X_matrix = []

    def create_phi_matrix(k, n, m):

        y_theta = train_y[k-n:k]  
        y_theta = y_theta[::-1]  

        u_theta = train_u[k-d-m:k-d+1] 
        u_theta = u_theta[::-1] 
    
        phi = np.concatenate((y_theta, u_theta))

        return phi

    for k in range(p, N):  
        phi_matrix = create_phi_matrix(k, n, m)
        X_matrix.append(phi_matrix) 

    X_matrix = np.array(X_matrix)

    return X_matrix


Creation of the Y matrix

In [3]:
def Y_matrix_creation(n,m,d,train_y):

    p = max(n,m+d)
    N = len(train_u)
    
    Y_matrix = []

    for k in range(p,N):
        y_p = np.array(train_y[k]) 
    
        Y_matrix = np.append(Y_matrix,y_p) 

    Y_matrix = np.array(Y_matrix).reshape(-1, 1) 

    return Y_matrix
    

Y test Generator

In [4]:
import numpy as np

def create_phi_test(i, n, d, m, y_test, test_u):

    phi = []

    y_end_index = i - 1
    y_initial_index = i - n

    x_end_index = i - d
    x_initial_index = i - d - m

    for j in range(y_initial_index, y_end_index + 1):
        if j > 0:
            phi.append(y_test[j])
        else:
            phi.append(0)

    phi = phi[::-1] 

    for j in range (x_end_index + 1, x_initial_index,-1):
        if j >= 0:
            phi.append(test_u[j])
        else:
            phi.append(0)

    return phi

def y_predict_generator(model, X_matrix, n, m, d, test_u):
    
    N = len(test_u)
    theta_matrix = model.coef_

    y_test = [] 

    for i in range(0,N-1):

        if i < d:
            y_test.append(0)
        else:
            phi_test_vector = create_phi_test(i, n, d, m, y_test, test_u)

            phi_test_matrix = np.array(phi_test_vector).reshape(1, -1)

            y_pred = np.dot(phi_test_matrix, theta_matrix.T) + model.intercept_

            y_test.append(float(y_pred[0]))
    
    return y_test[-400:]


Main 

In [ ]:
from sklearn.linear_model import Lasso, Ridge

n = 9
m = 9
d = 6
p = max(n, m, d)
N = len(test_u)

# X and Y matrix creation
X_matrix = X_matrix_creation(n, m, d, train_u, train_y)
Y_matrix = Y_matrix_creation(n, m, d, train_y)

# lasso 
lasso_model = Lasso(alpha=0.000000001).fit(X_matrix, Y_matrix)
y_lasso = y_predict_generator(lasso_model, X_matrix, n, m, d, test_u)

print(len(test_u))
print(len(y_lasso))

np.save('y_test.npy', y_lasso)
